In [2]:
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.constraints import unitnorm
from keras.layers.core import Reshape, Flatten, Merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from sklearn.cross_validation import KFold
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
import numpy as np
from sklearn import cross_validation
import math
from keras_input_data import make_idx_data
from load_vai import loadVAI
import _pickle as cPickle
from metrics import continuous_metrics




def lstm_cnn(W):
    
    nb_filter = 15
    filter_length = 30
    pool_length = 6
    
    region_input = Input(shape=(maxlen,), dtype='int32', name='region_input')
        ###这是一个逗号标志的输入的区域的句子，属于整个文章的一个区域。
    x = Embedding(W.shape[0], W.shape[1], weights=[W], input_length=maxlen)(region_input)

    lstm_output = LSTM(64, return_sequences=True, name='lstm')(x)  

    region_conv = Convolution1D(nb_filter=nb_filter,
                                    filter_length=filter_length,
                                    border_mode='valid',
                                    activation='relu',
                                    subsample_length=1)(lstm_output)
    region_max = MaxPooling1D(pool_length=pool_length)(region_conv)
    region_vector = Flatten()(region_max)
    textvector = Dense(64, activation='relu')(region_vector)
    predictions = Dense(1, activation='linear')(textvector)
    final_model = Model(region_input, predictions, name='model')
    model=final_model

    return model


if __name__ == '__main__':
    x = cPickle.load(open("mr.p", "rb"))
    revs, W, W2, word_idx_map, vocab = x[0], x[1], x[2], x[3], x[4]
    print("data loaded!")
    sentences=[]
    for rev in revs:
        sentence = rev['text']
        sentences.append(sentence)
    idx_data = make_idx_data(sentences, word_idx_map)
    #print(idx_data)

    dim = 'V'
    column = loadVAI(dim)
    irony=column
    maxlen = 87  # cut texts after this number of words (among top max_features most common words)
    batch_size = 8

    Y = np.array(irony)
    Y = [float(x) for x in Y]

    # X_train, X_test, y_train, y_test = cross_validation.train_test_split(idx_data, Y, test_size=0.2,
    #                                                                      random_state=2)
    n_MAE=0
    n_Pearson_r=0
    n_Spearman_r=0
    n_MSE=0
    n_R2=0
    n_MSE_sqrt=0
    SEED = 42
    n = 5  # repeat the CV procedure 5 times to get more precise results
    for i in range(n):
        # for each iteration, randomly hold out 20% of the data as CV set
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(
            idx_data, Y, test_size=.20, random_state=i * SEED)

        print(len(X_train), 'train sequences')
        print(len(X_test), 'test sequences')

        max_features = W.shape[0]  # shape of W: (13631, 300) , changed to 14027 through min_df = 3
        # print(max_features)

        print("Pad sequences (samples x time)")
        X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
        X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
        print('X_train shape:', X_train.shape)
        print('X_test shape:', X_test.shape)

    
        model =lstm_cnn(W)
        print('-------------now is lstm_cnn---------')
        model.compile(loss='mse', optimizer='adagrad')  # loss function: mse
        print("Train...")
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        result = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10,validation_data=(X_test, y_test),
                           callbacks=[early_stopping])
        score = model.evaluate(X_test, y_test, batch_size=batch_size)
        print('Test score:', score)
        # experiment evaluated by multiple metrics
        predict = model.predict(X_test, batch_size=batch_size).reshape((1, len(X_test)))[0]
        #print(predict)
        # print('Y_test: %s' % str(y_test))
        # print('Predict value: %s' % str(predict))
#         estimate=continuous_metrics(y_test, predict, 'prediction result:')
#         # MSE, MAE, Pearson_r, R2, Spearman_r, MSE_sqrt
      
#         n_MAE += estimate[0]
#         n_Pearson_r += estimate[1]
       
#     ndigit=3

#     avg_MAE =  round(n_MAE/5, ndigit)
#     avg_Pearson_r =  round(n_Pearson_r/5, ndigit)
 
#     print('average evaluate result:')
#     print(avg_MAE ,avg_Pearson_r)

data loaded!
 ----------------now is arousal-----------
803 train sequences
201 test sequences
Pad sequences (samples x time)
X_train shape: (803, 87)
X_test shape: (201, 87)
-------------now is lstm_cnn---------
Train...
Train on 803 samples, validate on 201 samples
Epoch 1/10
803/803 [==============================] - 12s - loss: 3.8513 - val_loss: 2.2600
Epoch 2/10
803/803 [==============================] - 12s - loss: 0.8591 - val_loss: 1.2096
Epoch 3/10
803/803 [==============================] - 12s - loss: 0.6373 - val_loss: 0.7199
Epoch 4/10
803/803 [==============================] - 12s - loss: 0.4745 - val_loss: 0.7430
Epoch 5/10
803/803 [==============================] - 12s - loss: 0.3635 - val_loss: 0.7920
Epoch 6/10
803/803 [==============================] - 12s - loss: 0.2564 - val_loss: 0.7596
Epoch 7/10
803/803 [==============================] - 12s - loss: 0.1909 - val_loss: 0.7802
Epoch 8/10
803/803 [==============================] - 12s - loss: 0.1514 - val_loss: 0.8